In [1]:
import pandas as pd
import numpy as np
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score


In [2]:
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/bagging/*')):
    print('file path',feat)
    a,b,c = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(c)
# load y
train = pd.read_csv("../input/new_data/train_set.csv")
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/bagging/rnn_model_bagging1.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging2.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging3.pkl
(56590, 19) (56590, 19)
(56590, 57)


In [3]:
# y=(train["class"]-1).astype(int)
# from keras.utils import np_utils
train_y=b

In [12]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat, average='macro'), True

In [10]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred, val_pred = np.zeros((102277,20)), np.zeros((102277,20))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_f1_l,all_val_f1_l = 0,0
    
    for i in range(19):
        val_loss_l,train_loss_l = 0,0
        val_f1_l,train_f1_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': 4,
                    #'scale_pos_weight':0.9,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 100
            else:
                s_round = 50
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      feval=lgb_f1_score,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=1)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            train_pred = model.predict(curr_x)
            val_pred[test_index, i] = model.predict(hold_out_x)
            curr_train_loss = log_loss(curr_y[:,i],train_pred)
            curr_val_loss = log_loss(hold_out_y[:,i],val_pred[test_index, i])

            curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
            curr_val_f1 = self_f1_score(hold_out_y[:,i],val_pred[test_index, i])

            print('ls',curr_train_loss,curr_val_loss,'f1',curr_train_f1,curr_val_f1)
            val_loss_l += curr_val_loss
            train_loss_l += curr_train_loss
            val_f1_l += curr_val_f1
            train_f1_l += curr_train_f1
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_f1_l = train_f1_l/k
        val_f1_l = val_f1_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class f1 train',train_f1_l,'f1 val',val_f1_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/19
        all_val_loss_l += val_loss_l/19
        all_train_f1_l += train_f1_l/19
        all_val_f1_l += val_f1_l/19
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all f1 avg',all_train_f1_l,all_val_f1_l)
    print('=======================================================')
    return val_pred, test_pred

print('done')

done


In [11]:
lgb_train, lgb_res = simple_ens('lgb',10,233,0.01,0.6)
# all loss avg 0.04155834441198802 0.05109057227649996
# all f1 avg 0.9119915361692824 0.8960796040017875

[1]	training's binary_logloss: 0.684293	valid_1's binary_logloss: 0.684348
Training until validation scores don't improve for 50 rounds.
[2]	training's binary_logloss: 0.675643	valid_1's binary_logloss: 0.675759
[3]	training's binary_logloss: 0.667084	valid_1's binary_logloss: 0.667246
[4]	training's binary_logloss: 0.658699	valid_1's binary_logloss: 0.658893
[5]	training's binary_logloss: 0.651375	valid_1's binary_logloss: 0.65162
[6]	training's binary_logloss: 0.643283	valid_1's binary_logloss: 0.643574
[7]	training's binary_logloss: 0.635344	valid_1's binary_logloss: 0.635682
[8]	training's binary_logloss: 0.627597	valid_1's binary_logloss: 0.627984
[9]	training's binary_logloss: 0.619987	valid_1's binary_logloss: 0.620422
[10]	training's binary_logloss: 0.612483	valid_1's binary_logloss: 0.612971
[11]	training's binary_logloss: 0.605119	valid_1's binary_logloss: 0.605641
[12]	training's binary_logloss: 0.597886	valid_1's binary_logloss: 0.598455
[13]	training's binary_logloss: 0.59

[119]	training's binary_logloss: 0.225433	valid_1's binary_logloss: 0.229863
[120]	training's binary_logloss: 0.223879	valid_1's binary_logloss: 0.228338
[121]	training's binary_logloss: 0.222345	valid_1's binary_logloss: 0.226832
[122]	training's binary_logloss: 0.220828	valid_1's binary_logloss: 0.225349
[123]	training's binary_logloss: 0.219331	valid_1's binary_logloss: 0.223877
[124]	training's binary_logloss: 0.217858	valid_1's binary_logloss: 0.222442
[125]	training's binary_logloss: 0.216396	valid_1's binary_logloss: 0.221011
[126]	training's binary_logloss: 0.214953	valid_1's binary_logloss: 0.219596
[127]	training's binary_logloss: 0.213541	valid_1's binary_logloss: 0.21821
[128]	training's binary_logloss: 0.212141	valid_1's binary_logloss: 0.216842
[129]	training's binary_logloss: 0.21075	valid_1's binary_logloss: 0.215478
[130]	training's binary_logloss: 0.209579	valid_1's binary_logloss: 0.214336
[131]	training's binary_logloss: 0.208231	valid_1's binary_logloss: 0.213014
[

[238]	training's binary_logloss: 0.127207	valid_1's binary_logloss: 0.134236
[239]	training's binary_logloss: 0.126809	valid_1's binary_logloss: 0.133856
[240]	training's binary_logloss: 0.126426	valid_1's binary_logloss: 0.133499
[241]	training's binary_logloss: 0.126039	valid_1's binary_logloss: 0.133132
[242]	training's binary_logloss: 0.12566	valid_1's binary_logloss: 0.132772
[243]	training's binary_logloss: 0.125282	valid_1's binary_logloss: 0.132412
[244]	training's binary_logloss: 0.124906	valid_1's binary_logloss: 0.13205
[245]	training's binary_logloss: 0.124535	valid_1's binary_logloss: 0.131702
[246]	training's binary_logloss: 0.124166	valid_1's binary_logloss: 0.131354
[247]	training's binary_logloss: 0.123802	valid_1's binary_logloss: 0.130999
[248]	training's binary_logloss: 0.123447	valid_1's binary_logloss: 0.130667
[249]	training's binary_logloss: 0.123092	valid_1's binary_logloss: 0.130333
[250]	training's binary_logloss: 0.122741	valid_1's binary_logloss: 0.130003
[

[348]	training's binary_logloss: 0.102039	valid_1's binary_logloss: 0.110721
[349]	training's binary_logloss: 0.101919	valid_1's binary_logloss: 0.110614
[350]	training's binary_logloss: 0.101802	valid_1's binary_logloss: 0.110509
[351]	training's binary_logloss: 0.101684	valid_1's binary_logloss: 0.1104
[352]	training's binary_logloss: 0.101567	valid_1's binary_logloss: 0.110295
[353]	training's binary_logloss: 0.101452	valid_1's binary_logloss: 0.11019
[354]	training's binary_logloss: 0.101338	valid_1's binary_logloss: 0.110082
[355]	training's binary_logloss: 0.101225	valid_1's binary_logloss: 0.109979
[356]	training's binary_logloss: 0.101113	valid_1's binary_logloss: 0.109881
[357]	training's binary_logloss: 0.101002	valid_1's binary_logloss: 0.109786
[358]	training's binary_logloss: 0.100892	valid_1's binary_logloss: 0.109685
[359]	training's binary_logloss: 0.100785	valid_1's binary_logloss: 0.109593
[360]	training's binary_logloss: 0.100676	valid_1's binary_logloss: 0.109497
[3

[459]	training's binary_logloss: 0.0939927	valid_1's binary_logloss: 0.10405
[460]	training's binary_logloss: 0.0939512	valid_1's binary_logloss: 0.104027
[461]	training's binary_logloss: 0.0939089	valid_1's binary_logloss: 0.104006
[462]	training's binary_logloss: 0.0938678	valid_1's binary_logloss: 0.10397
[463]	training's binary_logloss: 0.0938286	valid_1's binary_logloss: 0.103944
[464]	training's binary_logloss: 0.0937883	valid_1's binary_logloss: 0.103919
[465]	training's binary_logloss: 0.0937469	valid_1's binary_logloss: 0.103889
[466]	training's binary_logloss: 0.0937018	valid_1's binary_logloss: 0.103853
[467]	training's binary_logloss: 0.0936579	valid_1's binary_logloss: 0.103819
[468]	training's binary_logloss: 0.0936332	valid_1's binary_logloss: 0.103807
[469]	training's binary_logloss: 0.0935957	valid_1's binary_logloss: 0.103777
[470]	training's binary_logloss: 0.0935567	valid_1's binary_logloss: 0.103753
[471]	training's binary_logloss: 0.0935198	valid_1's binary_loglos

...

[93]	training's binary_logloss: 0.273973	valid_1's binary_logloss: 0.273776
[94]	training's binary_logloss: 0.271818	valid_1's binary_logloss: 0.271607
[95]	training's binary_logloss: 0.269702	valid_1's binary_logloss: 0.269493
[96]	training's binary_logloss: 0.267613	valid_1's binary_logloss: 0.267407
[97]	training's binary_logloss: 0.265539	valid_1's binary_logloss: 0.265327
[98]	training's binary_logloss: 0.263493	valid_1's binary_logloss: 0.263272
[99]	training's binary_logloss: 0.261473	valid_1's binary_logloss: 0.261247
[100]	training's binary_logloss: 0.259479	valid_1's binary_logloss: 0.25925
[101]	training's binary_logloss: 0.25751	valid_1's binary_logloss: 0.257276
[102]	training's binary_logloss: 0.255566	valid_1's binary_logloss: 0.255327
[103]	training's binary_logloss: 0.253654	valid_1's binary_logloss: 0.253421
[104]	training's binary_logloss: 0.251759	valid_1's binary_logloss: 0.251522
[105]	training's binary_logloss: 0.249889	valid_1's binary_logloss: 0.249649
[106]	tr

[209]	training's binary_logloss: 0.14128	valid_1's binary_logloss: 0.141608
[210]	training's binary_logloss: 0.140728	valid_1's binary_logloss: 0.141056
[211]	training's binary_logloss: 0.14019	valid_1's binary_logloss: 0.140516
[212]	training's binary_logloss: 0.139651	valid_1's binary_logloss: 0.139981
[213]	training's binary_logloss: 0.139118	valid_1's binary_logloss: 0.139451
[214]	training's binary_logloss: 0.138598	valid_1's binary_logloss: 0.138939
[215]	training's binary_logloss: 0.138085	valid_1's binary_logloss: 0.138431
[216]	training's binary_logloss: 0.13757	valid_1's binary_logloss: 0.137919
[217]	training's binary_logloss: 0.137062	valid_1's binary_logloss: 0.137414
[218]	training's binary_logloss: 0.136565	valid_1's binary_logloss: 0.136915
[219]	training's binary_logloss: 0.136067	valid_1's binary_logloss: 0.136422
[220]	training's binary_logloss: 0.135688	valid_1's binary_logloss: 0.136049
[221]	training's binary_logloss: 0.1352	valid_1's binary_logloss: 0.135562
[222

[322]	training's binary_logloss: 0.106286	valid_1's binary_logloss: 0.107667
[323]	training's binary_logloss: 0.106167	valid_1's binary_logloss: 0.107569
[324]	training's binary_logloss: 0.106009	valid_1's binary_logloss: 0.107419
[325]	training's binary_logloss: 0.105853	valid_1's binary_logloss: 0.107278
[326]	training's binary_logloss: 0.105698	valid_1's binary_logloss: 0.107136
[327]	training's binary_logloss: 0.105546	valid_1's binary_logloss: 0.107004
[328]	training's binary_logloss: 0.105395	valid_1's binary_logloss: 0.106866
[329]	training's binary_logloss: 0.105246	valid_1's binary_logloss: 0.106732
[330]	training's binary_logloss: 0.105097	valid_1's binary_logloss: 0.1066
[331]	training's binary_logloss: 0.104952	valid_1's binary_logloss: 0.106475
[332]	training's binary_logloss: 0.104808	valid_1's binary_logloss: 0.106344
[333]	training's binary_logloss: 0.104665	valid_1's binary_logloss: 0.106218
[334]	training's binary_logloss: 0.104523	valid_1's binary_logloss: 0.10609
[3

[434]	training's binary_logloss: 0.0956868	valid_1's binary_logloss: 0.0987923
[435]	training's binary_logloss: 0.0956352	valid_1's binary_logloss: 0.098755
[436]	training's binary_logloss: 0.0955825	valid_1's binary_logloss: 0.0987161
[437]	training's binary_logloss: 0.0955314	valid_1's binary_logloss: 0.0986804
[438]	training's binary_logloss: 0.0954804	valid_1's binary_logloss: 0.0986449
[439]	training's binary_logloss: 0.095431	valid_1's binary_logloss: 0.0986147
[440]	training's binary_logloss: 0.0953995	valid_1's binary_logloss: 0.0985858
[441]	training's binary_logloss: 0.09535	valid_1's binary_logloss: 0.0985456
[442]	training's binary_logloss: 0.0953018	valid_1's binary_logloss: 0.0985127
[443]	training's binary_logloss: 0.0952524	valid_1's binary_logloss: 0.0984825


KeyboardInterrupt: 

In [23]:
lgb_res[0]

array([3.34281403e-03, 1.05262902e-03, 2.65139344e-03, 3.28427221e-03,
       9.56538550e-01, 2.02733615e-03, 2.60089867e-04, 4.11558640e-04,
       2.56678473e-04, 1.60663793e-03, 5.55739527e-03, 5.49251895e-03,
       4.32519391e-04, 9.59766293e-04, 7.54894995e-04, 2.76494966e-04,
       3.73595172e-04, 1.04897244e-03, 6.43206971e-04, 0.00000000e+00])

In [24]:
name = ["class_prob_%s"%i for i in range(1,21)]
df_lgb = pd.DataFrame(lgb_res, columns=name)
df_lgb = df_lgb.drop('class_prob_20', axis=1)
df_lgb.to_csv('../pro/stacking_lgb_bagging.csv', index=False)